In [1]:
import py3Dmol as p3d
import numpy as np
import pickle
#from ase.build.rotate import minimize_rotation_and_translation as transf
#from ase.io import read
#import torch

In [2]:
def get_beta_schedule(beta_schedule, *, beta_start, beta_end, num_diffusion_timesteps):
    def sigmoid(x):
        return 1 / (np.exp(-x) + 1)

    if beta_schedule == "quad":
        betas = (
            np.linspace(
                beta_start**0.5,
                beta_end**0.5,
                num_diffusion_timesteps,
                dtype=np.float64,
            )
            ** 2
        )
    elif beta_schedule == "linear":
        betas = np.linspace(
            beta_start, beta_end, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "const":
        betas = beta_end * np.ones(num_diffusion_timesteps, dtype=np.float64)
    elif beta_schedule == "jsd":  # 1/T, 1/(T-1), 1/(T-2), ..., 1
        betas = 1.0 / np.linspace(
            num_diffusion_timesteps, 1, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "sigmoid":
        betas = np.linspace(-6, 6, num_diffusion_timesteps)
        betas = sigmoid(betas) * (beta_end - beta_start) + beta_start
    else:
        raise NotImplementedError(beta_schedule)
    assert betas.shape == (num_diffusion_timesteps,)
    return betas

def make_xyz(symbols, pos_array):
    msg = f"{len(pos_array)}\n"
    symbol_dict = {1:"H", 6:"C", 7:"N", 8:"O"}
    for s,xyz in zip(symbols, pos_array):
        if not isinstance(s, str):
            s = symbol_dict[s]
        msg += f"\n{s}"
        for v in xyz:
            msg += f" {v}"
    return msg

In [3]:
with open("tmp.xyz","r") as f: tmpxyz = f.read()

In [4]:
view = p3d.view(width=800, height=400, linked=True)
view.addModel(tmpxyz, "xyz",)
#view.addModel(xyz_ts, "xyz", viewer=(0,1))
#view.addModel(xyz_gen, "xyz", viewer=(0,2))
#view.addModel(xyz_p, "xyz", viewer=(0,3))
view.setStyle({'stick':{}})
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
# Animate optimization
viewer = p3d.view(width=1000, height=500)
viewer.addModelsAsFrames(xyz_traj)
viewer.animate({"loop": "forward"})
viewer.setStyle({"stick": {}})
viewer.rotate(-60, "x")
viewer.rotate(40, "y")
viewer.show()

NameError: name 'xyz_traj' is not defined

In [21]:
# Animate optimization

viewer = p3d.view(width=1000, height=500)
viewer.addModelsAsFrames(xyz_traj)
viewer.animate({"loop": "forward"})
viewer.setStyle({"stick": {}})
viewer.rotate(-60, "x")
viewer.rotate(40, "y")
viewer.show()


NameError: name 'xyz_traj' is not defined

In [20]:
!jupyter labextension install jupyterlab_3dmol

An error occured.
PermissionError: [Errno 13] Permission denied: '/appl/anaconda3/share/jupyter/lab/extensions'
See the log file for details:  /tmp/jupyterlab-debug-nf5toou4.log
